In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pwd
#%cd ./drive/My Drive/Colab Notebooks/UniverseDacon/
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
sample_submission = pd.read_csv('sample_submission.csv', index_col=0)
sample_submission.describe()

/content/drive/My Drive/Colab Notebooks/UniverseDacon


,STAR_WHITE_DWARF,STAR_CATY_VAR,STAR_BROWN_DWARF,SERENDIPITY_RED,REDDEN_STD,STAR_BHB,GALAXY,SERENDIPITY_DISTANT,QSO,SKY,STAR_RED_DWARF,ROSAT_D,STAR_PN,SERENDIPITY_FIRST,STAR_CARBON,SPECTROPHOTO_STD,STAR_SUB_DWARF,SERENDIPITY_MANUAL,SERENDIPITY_BLUE
count,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0,10009.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
# TRAIN 데이터의 TYPE을 SAMPLE_SUBMISSION에 대응하는 가변수 형태로 변환시킵니다.
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [0]:
y_train = train['type_num']
X_train = train.drop(columns=['id','type','type_num','fiberID'], axis=1)
test = test.drop(columns=['fiberID','id'])

In [0]:
# from sklearn.model_selection import train_test_split
# X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, \
#                                                   test_size=0.2, random_state=0)

### XGB, RF 

In [0]:
%%time
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
# 10, 1500 20, 2000 --> 0.395
# 10, 1300, 20, 1000 --> 0.3740680522180537(valid)
XGB_clf = XGBClassifier(max_depth = 12,n_esetimators=1300, n_jobs=-1, random_state=0)
XGB_clf.get_params()

rf_clf = RandomForestClassifier(max_depth=20, n_estimators=2000,max_features=7, oob_score=False, n_jobs=-1)

CPU times: user 333 µs, sys: 0 ns, total: 333 µs
Wall time: 344 µs


### LGBM

In [0]:
from lightgbm import LGBMClassifier

lgbm_wrapper = LGBMClassifier(boosting_type='gbdt', num_leaves=250, max_depth=-1, learning_rate=0.005, min_data_in_leaf = 5,
                             n_estimators=10000, subsample_for_bin=250000, objective='multiclass', min_split_gain=0.34, reg_alpha = 0.01, reg_lambda = 0.01,
                             min_child_weight=0.005, min_child_samples=17,n_jobs=-1, random_state=123)


In [0]:
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators=[('RF', rf_clf), ('lgbm', lgbm_wrapper)],
    voting='soft')

voting_clf.fit(X_train, y_train)

### Valid set Loss

In [0]:
yk_grd_probs = voting_clf.predict_proba(X_valid)
print(log_loss(y_valid, yk_grd_probs))

### Submission

In [0]:
voting_pred = voting_clf.predict_proba(test)
submission = pd.DataFrame(data=voting_pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('voting1.csv', index=True)
submission.describe()